In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('E:\\Test\\data\\pq\\green\\*\\*')

In [3]:
df_green.registerTempTable('green')

C:\tools\spark-3.3.2-bin-hadoop3\python\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [4]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [5]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('E:\\Test\\data\\report\\revenue\\green', mode='overwrite')

In [6]:
df_yellow = spark.read.parquet('E:\\Test\\data\\pq\\yellow\\*\\*')
df_yellow.registerTempTable('yellow')

In [7]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [8]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('E:\\Test\\data\\report\\revenue\\yellow', mode='overwrite')

In [9]:
df_green_revenue = spark.read.parquet('E:\\Test\\data\\report\\revenue\\green')
df_yellow_revenue = spark.read.parquet('E:\\Test\\data\\report\\revenue\\yellow')

In [10]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [11]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [12]:
df_join.show()

+-------------------+----+-----------------+--------------------+------------------+---------------------+
|               hour|zone|     green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+-----------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  13|             null|                null|1214.8000000000002|                   56|
|2020-01-01 00:00:00|  18|              7.8|                   1|               5.8|                    1|
|2020-01-01 00:00:00|  83|94.09999999999998|                   7|               9.8|                    1|
|2020-01-01 00:00:00| 114|             null|                null| 6256.429999999997|                  333|
|2020-01-01 00:00:00| 126|             null|                null|             170.6|                    2|
|2020-01-01 00:00:00| 134|            69.05|                   6|              null|                 null|
|2020-01-01 00:00:00| 161|           

In [13]:
df_join.write.parquet('E:\\Test\\data\\report\\revenue\\total', mode='overwrite')

In [16]:
df_join = spark.read.parquet('E:\\Test\\data\\report\\revenue\\total')

In [17]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [21]:
df_zones = spark.read.csv('taxi_zone_lookup.csv')
df_zones.schema

StructType([StructField('_c0', StringType(), True), StructField('_c1', StringType(), True), StructField('_c2', StringType(), True), StructField('_c3', StringType(), True)])

In [20]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

AttributeError: 'DataFrame' object has no attribute 'LocationID'

In [ ]:
df_result.drop('LocationID', 'zone').write.parquet('E:\\Test\\tmp\\revenue-zones')